In [10]:
import cv2
import numpy as np
import imutils

In [45]:
def tag_detection(img):
    
    # Convert from BGR to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Get the saturation plane - all black/white/gray pixels are zero, and colored pixels are above zero.
    s = hsv[:, :, 1]

    # detect yellow
    lower_range = np.array([25, 50, 70])
    upper_range = np.array([35, 255, 255])
    mask = cv2.inRange(hsv, lower_range, upper_range)

    # Apply threshold on s - use automatic threshold algorithm (use THRESH_OTSU).
    ret, thresh = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # Find contours in thresh (find the triangles).
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2]  # [-2] indexing takes return value before last (due to OpenCV compatibility issues).
    font = cv2.FONT_HERSHEY_COMPLEX
    
    areaList = []
    outputList = []
            
    # Iterate triangle contours
    for c in cnts:
        approx = cv2.approxPolyDP(c, 0.05*cv2.arcLength(c, True), True)
    #     cv2.drawContours(img, [approx], 0, (0), 5)
        x = approx.ravel()[0]
        y = approx.ravel()[1]

        if (len(approx) == 4):  #  Ignore very small contours
            # Mark rectangle with green line
            cv2.drawContours(img, [approx], 0, (0, 255, 0), 1)
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(approx)
            ar = float(w)/h
            if (not(ar >= 0.95 and ar <= 1.05)) and ar>3:
                areaList.append(cv2.contourArea(approx))
                print("crop")
                # Crop the bounding rectangle out of img
                out = img[y-20:y+h+20, x-20:x+w+20].copy()
                outputList.append(out)
                
    max_value = max(areaList)
    max_index = areaList.index(max_value)
    print(max_index)
    
    return outputList[max_index]
        

In [46]:
img = cv2.imread('w.jpg')
imgNew = tag_detection(img)
cv2.imshow("Crop", imgNew)
cv2.waitKey(0)
cv2.destroyAllWindows()

crop
0
